___
# Supply Planning - Summary


___
# Literature on Supply Planning / MRP

The goal of using an MRP strategy is to run your operations in a manner which improves your customer response without over
producing and having a lot of extra stock.

Two main objectives that MRP strategies try to solve are inventory reduction or lead time reduction.

One way to classifiy the materials you use for MRP is : 1) Cheap materials (Use consumption based methods - reorder point planning) 2)Long lead time materials or expensive materials needs other starategies

What do you need to consider while choosing a strategy :

1. Level of the BOM where demand management is done
    - Finished Goods - Component Levele - Phantom level

2. What point to produce final assembly
    - Make to stock - Make to order
    
3. What point yo procure components
    - Component in stock - Purchase stock after sale
  
4. Should current stock levels influence my production and procurement

In The below planning process we formulate a Net Requirements Planning from distributor point of view which assumes unconstraint requirements generation.

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from dateutil.relativedelta import relativedelta
pd.options.display.float_format = '{:,.2f}'.format
%matplotlib inline

### Loading the results of planning from Supply Planning Sheet

In [27]:
data = pd.read_csv("result.csv")

In [28]:
data.head()

,Plan,SKU,Date,Agency,Brand,Desc...4,Desc...5,Super.Category,Literage,Pack,...,Avg.CA,SD.Stocks,StockDays,OOS.CA,Time.Fence,Year,Month,Excess.CA,Scheduled.CA,ABClass
0,Region 1,Product 1002,2021-04-01,Supplier 89,Band 216,Desc 1002,Desc 1013,Category 2,4.50,6,...,0.67,1.16,0.00,0.00,2021-09-01,2021,Apr,0.00,0.00,C
1,Region 1,Product 1002,2021-05-01,Supplier 89,Band 216,Desc 1002,Desc 1013,Category 2,4.50,6,...,1.00,1.73,0.00,0.00,2021-09-01,2021,May,0.00,0.00,C
2,Region 1,Product 1002,2021-06-01,Supplier 89,Band 216,Desc 1002,Desc 1013,Category 2,4.50,6,...,1.67,2.89,0.00,0.00,2021-09-01,2021,Jun,0.00,0.00,C
3,Region 1,Product 1002,2021-07-01,Supplier 89,Band 216,Desc 1002,Desc 1013,Category 2,4.50,6,...,2.33,4.04,0.00,3.00,2021-09-01,2021,Jul,0.00,11.93,C
4,Region 1,Product 1002,2021-08-01,Supplier 89,Band 216,Desc 1002,Desc 1013,Category 2,4.50,6,...,2.33,4.04,0.00,2.00,2021-09-01,2021,Aug,0.00,1.42,C


## Lets summerize the data (Taking only the first 3 months Jul, Aug, Sep)

### The below shows the number of SKU's planned for each Region / Category / ABC Class

In [29]:
data['Date'] = pd.to_datetime(data['Date'])
current_date = datetime.today()
future_date = current_date + relativedelta(months=3)
datasummary = data[(data['Date'] < future_date) & (data['Date'] > current_date - relativedelta(days=16))] 
val = ['Forecast.CA','OOS.CA']
col = ['Year','Month']
idx = ['Plan','Super.Category','ABClass']
datasummary.pivot_table(values= 'SKU', columns=['Year','Month'], index=idx, aggfunc=lambda x: len(x.unique()), margins='all')

Year                             2021                     All
Month                             Aug   Jul   Oct   Sep      
Plan     Super.Category ABClass                              
Region 1 Category 1     A         104   104   104   104   104
                        B         115   115   115   115   115
                        C         285   285   285   285   285
         Category 2     A          61    61    61    61    61
                        B         114   114   114   114   114
                        C         494   494   494   494   494
         Category 3     A          10    10    10    10    10
                        B          16    16    16    16    16
                        C          68    68    68    68    68
         Category 4     A          11    11    11    11    11
                        B           8     8     8     8     8
                        C          22    22    22    22    22
Region 2 Category 1     A         226   226   226   226   226
                        B         317   317   317   317   317
                        C         841   841   841   841   841
         Category 2     A          82    82    82    82    82
                        B         110   110   110   110   110
                        C         457   457   457   457   457
         Category 3     A          23    23    23    23    23
                        B          39    39    39    39    39
                        C         125   125   125   125   125
         Category 4     A          16    16    16    16    16
                        B          20    20    20    20    20
                        C          60    60    60    60    60
Region 3 Category 1     A         114   114   114   114   114
                        B         186   186   186   186   186
                        C         368   368   368   368   368
         Category 2     A          36    36    36    36    36
                        B          71    71    71    71    71
                        C         381   381   381   381   381
         Category 3     A          21    21    21    21    21
                        B          25    25    25    25    25
                        C         122   122   122   122   122
         Category 4     A           8     8     8     8     8
                        B          10    10    10    10    10
                        C          13    13    13    13    13
Region 4 Category 1     A         147   147   147   147   147
                        B         237   237   237   237   237
                        C         590   590   590   590   590
         Category 2     A          48    48    48    48    48
                        B         125   125   125   125   125
                        C         633   633   633   633   633
         Category 3     A          13    13    13    13    13
                        B          32    32    32    32    32
                        C         129   129   129   129   129
         Category 4     A          13    13    13    13    13
                        B          10    10    10    10    10
                        C          17    17    17    17    17
All                              3810  3810  3810  3810  3810

## Now lets look at the summaries of Forecasted Volume and Out of STock Volume

In [34]:
val = ['Forecast.CA','OOS.CA']
col = ['Year','Month']
idx = ['Plan','Super.Category','ABClass']
summary1 = datasummary.pivot_table(values= val, columns=col, index=idx, aggfunc={'Forecast.CA':'sum','OOS.CA':'sum'}).reorder_levels([1,2,0], axis=1).sort_index(axis=1)
summary1

Year                                   2021                                \
Month                                   Aug                  Jul            
                                Forecast.CA   OOS.CA Forecast.CA   OOS.CA   
Plan     Super.Category ABClass                                             
Region 1 Category 1     A          1,910.00   645.92    1,267.00   252.42   
                        B            544.00   133.67      531.00   159.50   
                        C            639.00   152.75      595.00   160.37   
         Category 2     A         34,084.00 3,755.50   19,337.00 8,272.58   
                        B          8,465.00 1,618.00    7,076.00 2,491.67   
                        C          5,454.00 1,087.42    4,772.00 1,057.88   
         Category 3     A         13,057.00 1,656.12    8,332.00    99.00   
                        B          4,234.00 1,532.00    3,555.00   632.00   
                        C          4,688.00 3,168.96    3,587.00 2,470.00   
         Category 4     A            211.00    59.00      129.00    32.00   
                        B             42.00    11.67       34.00     1.08   
                        C             51.00     9.67       45.00     2.00   
Region 2 Category 1     A         40,651.00 1,578.92   26,015.46 7,548.43   
                        B          8,399.00   798.33    4,843.58   647.73   
                        C          4,837.00 1,287.25    3,784.20 1,178.59   
         Category 2     A         13,048.00   354.92    7,887.49   149.17   
                        B          2,833.00   761.55    1,937.82   380.67   
                        C          1,762.00   369.83    1,393.84   515.80   
         Category 3     A         28,303.00    84.00   23,984.71 2,734.25   
                        B         10,418.00 1,279.13    7,437.80 1,568.92   
                        C         11,947.00 1,103.63    3,167.68   802.41   
         Category 4     A          2,092.00     9.17    1,514.50     0.00   
                        B            406.00    18.58      369.75    11.25   
                        C            218.00    26.83      200.58    15.92   
Region 3 Category 1     A         16,828.00 2,089.92    8,013.45 1,074.01   
                        B          3,775.00   785.33    2,494.24   520.34   
                        C          1,636.00   512.25    1,253.01   653.17   
         Category 2     A         53,450.00 4,155.17   30,765.81     0.00   
                        B         10,166.00 2,297.08    5,602.19   420.84   
                        C          5,376.00 1,770.51    3,771.95 1,457.25   
         Category 3     A         76,995.00 9,965.17   33,866.18   700.00   
                        B         17,088.00 1,477.08    9,396.63   925.33   
                        C          7,855.00 1,156.46    4,916.76 1,529.98   
         Category 4     A            226.00    20.67      139.33    15.33   
                        B             87.00    53.33       58.17    48.00   
                        C             17.00     9.00       44.17     8.50   
Region 4 Category 1     A          8,240.00   304.00    5,782.02 1,006.58   
                        B          2,595.00   518.67    1,910.57   335.40   
                        C          1,905.00 1,119.25    1,737.09 1,179.42   
         Category 2     A         20,306.00 2,016.96   21,754.38 6,404.25   
                        B          4,492.00 1,141.58    5,307.81 1,824.55   
                        C          3,211.00 1,524.85    3,608.17 1,546.00   
         Category 3     A         30,710.00   454.08   41,335.54 9,049.96   
                        B         10,298.00 1,125.00   12,271.71 2,130.54   
                        C          5,214.00 1,420.50    4,772.21 2,239.96   
         Category 4     A            152.00    10.00       91.33    15.83   
                        B             60.00    19.83       45.17    15.67   
                        C             42.00    23.17       41.58    20.92

## Now lets look into the SKU's that have high out of stock volumes

In [35]:
#datasummary.groupby(['Plan','Super.Category','SKU','Year','Month'])['OOS.CA'].agg(np.sum).reset_index()
oos = datasummary.pivot_table(index=['Plan','Super.Category','SKU'], columns=['Month'], values='OOS.CA', aggfunc='sum').reset_index()
oos['Total'] = oos['Jul'] + oos['Aug'] + oos['Sep'] + oos['Oct']
oos.sort_values(['Plan','Total'],ascending=False).head()

Month,Plan,Super.Category,SKU,Aug,Jul,Oct,Sep,Total
6820,Region 4,Category 3,Product 28,0.00,"8,021.63","3,418.00",0.00,"11,439.63"
5997,Region 4,Category 2,Product 1081,820.00,"3,288.00",0.00,0.00,"4,108.00"
6834,Region 4,Category 3,Product 31,0.00,0.00,"2,400.00",0.00,"2,400.00"
6807,Region 4,Category 3,Product 23,316.00,520.00,516.00,321.00,"1,673.00"
6848,Region 4,Category 3,Product 35,451.00,410.00,0.00,483.00,"1,344.00"


### Lets look Region 4 SKU: Product 31	to understand why there is an availability issue

### The below explaning will give you an idea on how the horizontal plan was constructed for the SKU

#### Sales.CA is the month to date sales volume

![](Capture.PNG)

In [38]:
data['Date'] = pd.to_datetime(data['Date'])
current_date = datetime.today()
future_date = current_date + relativedelta(months=7)
datadetails = data[(data['Date'] < future_date) & (data['Date'] > current_date - relativedelta(days=16))] 
ids = ['Plan','SKU','LT','SD','ABClass','Date']
val = ['Beginning.Inv.CA','Transfer.CA','Purchase.CA','Planned.CA','Forecast.CA',
       'Sales.CA','Closing.inv.CA','OOS.CA','Exs.Consump.CA','Scheduled.CA','StockDays','Excess.CA']
datadetails = datadetails.melt(id_vars=ids, value_vars=val)
ids = ['Plan','SKU','LT','SD','ABClass','variable']
datadetails = datadetails.pivot_table(values='value', index=ids, columns = 'Date', aggfunc=np.sum)
a = datadetails.xs(['Region 4','Product 31'],level=['Plan','SKU']).reset_index()
sorter = dict(zip(val, range(len(val))))
a['Tm_Rank'] = a['variable'].map(sorter)
a.sort_values(['Tm_Rank'], ascending = True).drop('Tm_Rank', 1).set_index(['LT','SD','ABClass','variable'])

Date                             2021-07-01 00:00:00  2021-08-01 00:00:00  \
LT  SD ABClass variable                                                     
115 45 B       Beginning.Inv.CA             8,223.00             6,376.00   
               Transfer.CA                      0.00                 0.00   
               Purchase.CA                  4,782.00             5,203.00   
               Planned.CA                       0.00                 0.00   
               Forecast.CA                  6,629.00             3,500.00   
               Sales.CA                     1,305.00                 0.00   
               Closing.inv.CA               6,376.00             8,079.00   
               OOS.CA                           0.00                 0.00   
               Exs.Consump.CA                   0.00                 0.00   
               Scheduled.CA                10,752.50             3,002.50   
               StockDays                       38.50                52.01   
               Excess.CA                        0.00                 0.00   

Date                             2021-09-01 00:00:00  2021-10-01 00:00:00  \
LT  SD ABClass variable                                                     
115 45 B       Beginning.Inv.CA             8,079.00             3,304.00   
               Transfer.CA                      0.00                 0.00   
               Purchase.CA                      0.00                 0.00   
               Planned.CA                       0.00                 0.00   
               Forecast.CA                  4,775.00             5,704.00   
               Sales.CA                         0.00                 0.00   
               Closing.inv.CA               3,304.00                 0.00   
               OOS.CA                           0.00             2,400.00   
               Exs.Consump.CA                   0.00                 0.00   
               Scheduled.CA                 4,501.50             3,004.50   
               StockDays                       19.85                 0.00   
               Excess.CA                        0.00                 0.00   

Date                             2021-11-01 00:00:00  2021-12-01 00:00:00  \
LT  SD ABClass variable                                                     
115 45 B       Beginning.Inv.CA                 0.00             6,251.50   
               Transfer.CA                      0.00                 0.00   
               Purchase.CA                      0.00                 0.00   
               Planned.CA                  10,752.50             3,002.50   
               Forecast.CA                  4,501.00             3,501.00   
               Sales.CA                         0.00                 0.00   
               Closing.inv.CA               6,251.50             5,753.00   
               OOS.CA                           0.00                 0.00   
               Exs.Consump.CA                   0.00                 0.00   
               Scheduled.CA                 2,820.00             2,706.50   
               StockDays                       45.00                45.00   
               Excess.CA                        0.00                 0.00   

Date                             2022-01-01 00:00:00  2022-02-01 00:00:00  
LT  SD ABClass variable                                                    
115 45 B       Beginning.Inv.CA             5,753.00             5,753.50  
               Transfer.CA                      0.00                 0.00  
               Purchase.CA                      0.00                 0.00  
               Planned.CA                   4,501.50             3,004.50  
               Forecast.CA                  4,501.00             3,504.00  
               Sales.CA                         0.00                 0.00  
               Closing.inv.CA               5,753.50             5,254.00  
               OOS.CA                           0.00                 0.00  
   